In [1]:
!pip install datasets transformers optimum onnx onnxruntime torchaudio librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
!pip install --upgrade optimum

In [ ]:
from huggingface_hub import login
login("token")
MODEL_NAME = "openai/whisper-base"

In [5]:
SAVE_DIR_TEMPLATE = "optimized_whisper_tiny"

In [6]:
from pathlib import Path
from datasets import load_dataset
from itertools import islice
from transformers import (
    pipeline,
    AutoModelForSpeechSeq2Seq,
    AutoTokenizer,
    AutoFeatureExtractor,
)
from optimum.onnxruntime import ORTModelForSpeechSeq2Seq, ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig
from tqdm.notebook import tqdm
import time
import numpy as np
import psutil
import torch
import gc
from difflib import SequenceMatcher
import os

In [7]:
dataset = load_dataset("librispeech_asr", "clean", split="test", streaming=True, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

librispeech_asr.py:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

In [8]:
# Set sample size and warmup size
SAMPLE_SIZE = 100
WARMUP_SIZE = 10

# Take warmup + benchmark samples
samples_warmup = list(islice(dataset, WARMUP_SIZE))
samples_benchmark = list(islice(dataset, SAMPLE_SIZE))

print(f"Loaded {WARMUP_SIZE} warmup samples and {SAMPLE_SIZE} benchmark samples.")

Loaded 10 warmup samples and 100 benchmark samples.


In [9]:
# Prepare Whisper model
print("Loading Whisper model...")
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Load PyTorch model
model = AutoModelForSpeechSeq2Seq.from_pretrained(MODEL_NAME)

Loading Whisper model...


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

In [10]:
# Step 1: Export to ONNX (first save the model in Transformers format)
print("Exporting Whisper model to ONNX format...")

# Save the original model in Transformers format so Optimum can export it
model_output_dir = "whisper_tiny_transformers"
AutoModelForSpeechSeq2Seq.from_pretrained(MODEL_NAME).save_pretrained(model_output_dir)

Exporting Whisper model to ONNX format...


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [11]:
# Now use Optimum to export and optimize
print("Converting to ONNX and optimizing with optimization_level=99...")
ort_model = ORTModelForSpeechSeq2Seq.from_pretrained(model_output_dir, from_transformers=True)
optimizer = ORTOptimizer.from_pretrained(ort_model)
optimization_config = OptimizationConfig(optimization_level=99)
optimizer.optimize(save_dir=SAVE_DIR_TEMPLATE, optimization_config=optimization_config)

No ONNX files were found for whisper_tiny_transformers, setting `export=True` to convert the model to ONNX. Don't forget to save the resulting model with `.save_pretrained()`
The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead


Converting to ONNX and optimizing with optimization_level=99...


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/modeling_whisper.py:881: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_features.shape[-1] != expected_seq_length:
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/modeling_whisper.py:330: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz, self.num_heads, tgt_len, self.head_dim):
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead

PosixPath('optimized_whisper_tiny')

In [12]:
# Reload optimized model
print("Reloading optimized model...")
optimized_model = ORTModelForSpeechSeq2Seq.from_pretrained(SAVE_DIR_TEMPLATE)
pipe = pipeline(
    task="automatic-speech-recognition",
    model=optimized_model,
    tokenizer=tokenizer,
    feature_extractor=feature_extractor,
)


Reloading optimized model...


Device set to use cpu


In [ ]:
# A fix to an issue with forced_decoder_ids in generation_config
if hasattr(optimized_model.config, 'forced_decoder_ids'):
    optimized_model.config.forced_decoder_ids = None

if hasattr(optimized_model, 'generation_config'):
    if hasattr(optimized_model.generation_config, 'forced_decoder_ids'):
        optimized_model.generation_config.forced_decoder_ids = None

In [14]:
# Benchmarking function
def run_inference(samples, desc="Processing"):
    times = []
    references = []
    predictions = []

    for sample in tqdm(samples, desc=desc):
        audio = sample["audio"]["array"]
        reference = sample["text"].strip().lower()

        start_time = time.time()
        prediction = pipe(audio)["text"].strip().lower()
        end_time = time.time()

        times.append(end_time - start_time)
        references.append(reference)
        predictions.append(prediction)

    return times, predictions, references


In [15]:
# A fix to an issue mentioned before
if hasattr(ort_model.config, 'forced_decoder_ids'):
    ort_model.config.forced_decoder_ids = None

if hasattr(ort_model, 'generation_config'):
    if hasattr(ort_model.generation_config, 'forced_decoder_ids'):
        ort_model.generation_config.forced_decoder_ids = None

In [16]:
# Warmup phase
print("\nRunning warmup...")
run_inference(samples_warmup, desc="Warmup")

# Benchmark phase
print("\nRunning benchmark...")
times, predictions, references = run_inference(samples_benchmark, desc="Benchmark")

# Calculate Word Error Rate (WER)
def calculate_wer(references, predictions):
    wer_scores = []
    for ref, pred in zip(references, predictions):
        ref_words = ref.split()
        pred_words = pred.split()
        matcher = SequenceMatcher(None, ref_words, pred_words)
        opcodes = matcher.get_opcodes()
        errors = sum(
            max(len(ref_words[i1:i2]), len(pred_words[j1:j2])) if tag != "equal" else 0
            for tag, i1, i2, j1, j2 in opcodes
        )
        wer = errors / len(ref_words) if len(ref_words) > 0 else float("inf")
        wer_scores.append(wer)
    return np.mean(wer_scores)

# Compute metrics
avg_time = np.mean(times)
avg_wer = calculate_wer(references, predictions)
ram_usage = psutil.virtual_memory().used / (1024 ** 2)
model_size = sum(f.stat().st_size for f in Path(SAVE_DIR_TEMPLATE).rglob('*')) / (1024 ** 2)



Running warmup...


Warmup:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265


Running benchmark...


Benchmark:   0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
# Print results
print("\n--- Benchmark Results ---")
print(f"Average Inference Time: {avg_time:.2f} seconds")
print(f"Average WER: {avg_wer * 100:.2f}%")
print(f"RAM Usage: {ram_usage:.2f} MB")
print(f"Model Size: {model_size:.2f} MB")


--- Benchmark Results ---
Average Inference Time: 5.95 seconds
Average WER: 19.23%
RAM Usage: 7598.21 MB
Model Size: 666.12 MB
